## Additive Combinatorics Demo

Author: Cheuk Ting Li  

In [2]:
from psitip import *
PsiOpts.setting(solver = "pyomo.glpk")     # Set linear programming solver
PsiOpts.setting(repr_latex = True)         # Turn on Jupyter Notebook LaTeX display
PsiOpts.setting(venn_latex = True)         # Turn on LaTeX in diagrams
PsiOpts.setting(proof_note_color = "blue") # Reasons in proofs are blue
PsiOpts.setting(solve_display_reg = True)  # Display claims in solve commands

X, Y, Z, W = rv("X, Y, Z, W", alg="group") # Group-valued random variables
# Other valid choices are "semigroup", "abelian", "torsionfree" and "real"

In [2]:
# Capacity of additive noise channel
(indep(X, Z) >> (I(X & X*Z) == H(X*Z) - H(Z))).solve(full=True)

((  indep(X, Z)  )
>> (  H(Z)+I(X&X*Z) == H(X*Z)  )) is True


1. Steps: 
H(Z)
= H(Z|X)   (since indep(Z, X) )
= I(X*Z&Z|X)   (since H(Z|X*Z+X) == 0 )
= H(X*Z|X)   (since H(X*Z|X+Z) == 0 )

2. Steps: 
H(X*Z|X)
= I(X*Z&Z|X)   (since H(X*Z|X+Z) == 0 )
<= H(Z|X)
= H(Z)   (since indep(Z, X) )

In [8]:
# Entropy of sum is submodular [Madiman 2008]
# The inequality is automatically discovered by the program
indep(X, Y, Z).discover([H(Y), H(X*Y), H(Y*Z), H(X*Y*Z)])

( ( H(X*Y) <= H(X*Y*Z) )
 &( H(Y*Z) <= H(X*Y*Z) )
 &( H(X*Y*Z)+H(Y) <= H(X*Y)+H(Y*Z) ) )

In [9]:
# Output proof
(indep(X, Y, Z) >> (H(X*Y*Z) + H(Y) <= H(X*Y) + H(Y*Z))).solve(full=True)

((  indep(X, Y, Z)  )
>> (  H(X*Y*Z)+H(Y) <= H(X*Y)+H(Y*Z)  )) is True


H(X*Y*Z)+H(Y)
= H(X*Y*Z)+H(Y|Z)   (since indep(Z, Y) )
= H(X*Y*Z)+I(Y&Y*Z|Z)   (since H(Y|Y*Z+Z) == 0 )
= H(X*Y*Z)+H(Y*Z|Z)   (since H(Y*Z|Y+Z) == 0 )
<= I(X*Y*Z&Y*Z|Z)+H(X*Y*Z+Y*Z)
= H(Y*Z)+H(X*Y*Z|Z)   (since markov(X*Y*Z, Y*Z, Z) )
= H(Y*Z)+I(X*Y&X*Y*Z|Z)   (since H(X*Y*Z|X*Y+Z) == 0 )
<= H(Y*Z)+H(X*Y|Z)
= H(X*Y)+H(Y*Z)   (since indep(Z, X*Y) )

In [3]:
# Ruzsa triangle inequality [Ruzsa 1996], [Tao 2010]
# The inequality is automatically discovered by the program
indep(X, Y, Z).discover([H(X), H(Y), H(Z), H(X/Z), H(X/Y), H(Y/Z)])

( ( H(X) <= H(X*Y**-1) )
 &( H(X) <= H(X*Z**-1) )
 &( H(Y) <= H(X*Y**-1) )
 &( H(Y) <= H(Y*Z**-1) )
 &( H(Z) <= H(X*Z**-1) )
 &( H(Z) <= H(Y*Z**-1) )
 &( H(X*Y**-1) <= H(X)+H(Y) )
 &( H(X*Z**-1) <= H(X)+H(Z) )
 &( H(Y*Z**-1) <= H(Y)+H(Z) )
 &( H(X)+H(Y*Z**-1) <= H(X*Y**-1)+H(X*Z**-1) )
 &( H(X*Y**-1)+H(Z) <= H(X*Z**-1)+H(Y*Z**-1) )
 &( H(X*Z**-1)+H(Y) <= H(X*Y**-1)+H(Y*Z**-1) ) )

In [4]:
# Output proof
(indep(X, Y, Z) >> (H(X/Z) <= H(X/Y) + H(Y/Z) - H(Y))).solve(full=True)

((  indep(X, Y, Z)  )
>> (  H(Y)+H(X*Z**-1) <= H(X*Y**-1)+H(Y*Z**-1)  )) is True


H(Y)+H(X*Z**-1)
<= I(X*Z**-1&Z)+H(X*Z**-1+Y|Z)   (since indep(X*Z**-1+Z, Y) )
<= I(X*Z**-1+Y*Z**-1&Z)+H(X*Z**-1+Y|Z)
= H(X*Z**-1)+I(Y*Z**-1&Y+Z|X*Z**-1)   (since H(Y|Y*Z**-1+Z+X*Z**-1) == 0 )
= H(X*Z**-1+Y*Z**-1)   (since H(Y*Z**-1|Y+Z+X*Z**-1) == 0 )
<= H(X*Y**-1+X*Z**-1+Y*Z**-1)
= H(X*Y**-1+Y*Z**-1)   (since H(X*Z**-1|X*Y**-1+Y*Z**-1) == 0 )
<= H(X*Y**-1)+H(Y*Z**-1)

In [3]:
# The proof of the sum-difference inequality [Ruzsa 1996], [Tao 2010] is harder
# We use the construction of auxiliary random variables in [Tao 2010]

X, Y = rv("X, Y", alg="abelian")  # Abelian-group-valued random variables

X2, X3 = rv_array("X", 2, 4, alg="abelian")
Y2, Y3 = rv_array("Y", 2, 4, alg="abelian")

Y2 = X2 * Y / X   # since X/Y == X2/Y2

# Auxiliary construction
r = region(
    indep(X, Y), indep(X2, Y2), indep(X3, Y3, X+Y+X2+Y2),
    markov(X+Y, X/Y, X2+Y2),
    eqdist([X, Y, X/Y], [X2, Y2, X/Y], [X3, Y2, X3/Y2], [X, Y3, X/Y3])
)

# The remaining step is automatic, gives desired result
r.discover([H(X), H(Y), H(X/Y), H(X3*Y3)])

( ( H(X) <= H(X*Y**-1) )
 &( H(X) <= H(X_3*Y_3) )
 &( H(Y) <= H(X*Y**-1) )
 &( H(Y) <= H(X_3*Y_3) )
 &( H(X*Y**-1) <= H(X)+H(Y) )
 &( H(X_3*Y_3) <= H(X)+H(Y) )
 &( H(X)+H(X_3*Y_3)+H(Y) <= 3*H(X*Y**-1) ) )

In [5]:
# Output the proof
(r >> (H(X3*Y3) <= 3*H(X/Y) - H(X) - H(Y))).proof()

### References
- Madiman, Mokshay. "On the entropy of sums." 2008 IEEE Information Theory Workshop. IEEE, 2008.
- Ruzsa, Imre Z. "Sums of finite sets." Number Theory: New York Seminar 1991–1995. Springer, New York, NY, 1996.
- Tao, Terence. "Sumset and inverse sumset theory for Shannon entropy." Combinatorics, Probability and Computing 19.4 (2010): 603-639.